# intake sandbox // CCiA CMIP5 data export

>28 April 2021<br>
>Thomas Moore

In [1]:
import xarray as xr
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import numpy as np
import xrft
import scipy
%config Completer.use_jedi = False

# Spin up a little cluster with `dask-jobqueue`

### Specs for an individual worker

In [2]:
walltime = '04:00:00'
cores = 2
memory = '8GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),'-l mem='+str(memory),
                                '-l storage=gdata/v14+scratch/v14+gdata/lp01+scratch/lp01+scratch/ux06+gdata/rr3+gdata/al33'],
                     header_skip=["select"])

### Scale up to n workers

In [3]:
cluster.scale(jobs=4)

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://10.6.64.50:45879 Dashboard: http://10.6.64.50:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
client

Client Scheduler: tcp://10.6.64.50:45879 Dashboard: http://10.6.64.50:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Let's use Scott Wales'(CLEX) catalog?
> https://nbviewer.jupyter.org/gist/ScottWales/7402c7c5cbd96f6bed62f810899b611a

In [7]:
import intake
NCI_catalog = intake.open_catalog('/g/data/hh5/public/apps/nci-intake-catalogue/catalogue.yaml')
list(NCI_catalog)

['era5', 'era5_land', 'ecmwf', 'esgf']

In [8]:
list(NCI_catalog.esgf)

['cmip5',
 'cmip5_all',
 'cmip6',
 'cmip6_all',
 'cordex',
 'cordex_all',
 'cmip5_gr1p5',
 'cmip6_gr1p5']

# loading example

In [9]:
# manually check and enter # of ensembles as they vary across models
CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog = NCI_catalog.esgf.cmip5_all.search\
(institute="MOHC",variable=['psl','pr'],time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',ensemble='r1i1p1')
if CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('version')['version']['count'] == 1:
    CCiA_HadGEM2_CC_rcp85_r1i1p1_ds = xr.open_mfdataset(CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('path')['path']['values'])
else:
    print("there's more than one version so write some kind of logic here to deal with it")
CCiA_HadGEM2_CC_rcp85_r1i1p1_ds   

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 145, lon: 192, time: 1141)
Coordinates:
  * time       (time) object 2005-12-16 00:00:00 ... 2100-12-16 00:00:00
  * lat        (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(300, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(300, 145, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(300, 192, 2), meta=np.ndarray>
    pr         (time, lat, lon) float32 dask.array<chunksize=(300, 145, 192), meta=np.ndarray>
    psl        (time, lat, lon) float32 dask.array<chunksize=(300, 145, 192), meta=np.ndarray>
Attributes: (12/28)
    institution:            Met Office Hadley Centre, Fitzroy Road, Exeter, D...
    institute_id:           MOHC
    experiment_id:          rcp85
    source:                 HadGEM2-CC (2011) atmosphere: HadGAM2(N96L60); oc...
    model_id:               HadGEM2-CC
    forcing:                GHG, Oz, SA, LU, Sl, Vl, BC, OC, (GHG = CO2, N2O,...
    ...                     ...
    table_id:               Table Amon (26 July 2011) 976b7fd1d9e1be31dddd28f...
    title:                  HadGEM2-CC model output prepared for CMIP5 RCP8.5
    parent_experiment:      historical
    modeling_realm:         atmos
    realization:            1
    cmor_version:           2.7.1

# Build CCIA datasets for `pr` and `psl` variables
> ref: https://www.climatechangeinaustralia.gov.au/en/learning-support/faqs/eight-climate-models-data/

## Model 1 : CCiA_HadGEM2_CC

> make historical and rcp85 catalog objects for `pr` and `psl`

In [26]:
CCiA_HadGEM2_CC_rcp85_psl_catalog = NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable=['psl'],time_frequency='mon',experiment=['rcp85'],model='HadGEM2-CC')
CCiA_HadGEM2_CC_historical_psl_catalog = NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable=['psl'],time_frequency='mon',experiment=['historical'],model='HadGEM2-CC')

In [27]:
CCiA_HadGEM2_CC_historical_psl_catalog

,unique
path,10
project,1
product,1
institute,1
model,1
experiment,1
time_frequency,1
realm,1
cmor_table,1
ensemble,3


In [32]:
CCiA_HadGEM2_CC_historical_psl_path_list = CCiA_HadGEM2_CC_historical_psl_catalog.unique('path')['path']['values']
CCiA_HadGEM2_CC_historical_psl_path_list

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_185912-188411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_188412-190911.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_190912-193411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_193412-195911.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_198412-200511.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-C

> how many historical psl ensembles

In [37]:
CCiA_HadGEM2_CC_historical_psl_catalog.unique('ensemble')['ensemble']['count']

3

In [38]:
CCiA_HadGEM2_CC_historical_psl_catalog.unique('ensemble')['ensemble']['values']

['r1i1p1', 'r2i1p1', 'r3i1p1']

In [39]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r1i1p1').unique('version')

{'version': {'count': 1, 'values': ['v20110927']}}

In [40]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r1i1p1').unique('date_range')

{'date_range': {'count': 6,
  'values': ['185912-188411',
   '188412-190911',
   '190912-193411',
   '193412-195911',
   '195912-198411',
   '198412-200511']}}

In [41]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r2i1p1').unique('version')

{'version': {'count': 1, 'values': ['v20111129']}}

In [42]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r2i1p1').unique('date_range')

{'date_range': {'count': 2, 'values': ['195912-198411', '198412-200512']}}

In [43]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r3i1p1').unique('version')

{'version': {'count': 1, 'values': ['v20111208']}}

In [44]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(ensemble='r3i1p1').unique('date_range')

{'date_range': {'count': 2, 'values': ['195912-198411', '198412-200512']}}

In [81]:
CCiA_HadGEM2_CC_historical_psl_path_list = CCiA_HadGEM2_CC_historical_psl_catalog.search(date_range=['195912-198411','198412-200511', '198412-200512']).unique('path')['path']['values']
CCiA_HadGEM2_CC_historical_psl_path_list

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_198412-200511.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r2i1p1/v20111129/psl/psl_Amon_HadGEM2-CC_historical_r2i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r2i1p1/v20111129/psl/psl_Amon_HadGEM2-CC_historical_r2i1p1_198412-200512.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r3i1p1/v20111208/psl/psl_Amon_HadGEM2-CC_historical_r3i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r3i1p1/v20111208/psl/psl_Amon_HadGEM2-CC_historical_r3i1p1_198412-200512.nc']

In [82]:
CCiA_HadGEM2_CC_historical_psl_r1i1p1_path_list = CCiA_HadGEM2_CC_historical_psl_catalog.search(date_range=['195912-198411','198412-200511', '198412-200512'],ensemble='r1i1p1').unique('path')['path']['values']
CCiA_HadGEM2_CC_historical_psl_r1i1p1_path_list

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_198412-200511.nc']

In [83]:
CCiA_HadGEM2_CC_historical_psl_r2i1p1_path_list = CCiA_HadGEM2_CC_historical_psl_catalog.search(date_range=['195912-198411','198412-200511', '198412-200512'],ensemble='r2i1p1').unique('path')['path']['values']
CCiA_HadGEM2_CC_historical_psl_r2i1p1_path_list

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r2i1p1/v20111129/psl/psl_Amon_HadGEM2-CC_historical_r2i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r2i1p1/v20111129/psl/psl_Amon_HadGEM2-CC_historical_r2i1p1_198412-200512.nc']

In [84]:
CCiA_HadGEM2_CC_historical_psl_r3i1p1_path_list = CCiA_HadGEM2_CC_historical_psl_catalog.search(date_range=['195912-198411','198412-200511', '198412-200512'],ensemble='r3i1p1').unique('path')['path']['values']
CCiA_HadGEM2_CC_historical_psl_r3i1p1_path_list

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r3i1p1/v20111208/psl/psl_Amon_HadGEM2-CC_historical_r3i1p1_195912-198411.nc',
 '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r3i1p1/v20111208/psl/psl_Amon_HadGEM2-CC_historical_r3i1p1_198412-200512.nc']

# Calculate historical psl for HadGEM2_CC psl

In [85]:
CCiA_HadGEM2_CC_historical_psl_r1i1p1_ds = xr.open_mfdataset(CCiA_HadGEM2_CC_historical_psl_r1i1p1_path_list,parallel=True)
CCiA_HadGEM2_CC_historical_psl_r1i1p1_ds = CCiA_HadGEM2_CC_historical_psl_r1i1p1_ds.rename({'psl':'psl_r1i1p1'})

In [86]:
CCiA_HadGEM2_CC_historical_psl_r2i1p1_ds = xr.open_mfdataset(CCiA_HadGEM2_CC_historical_psl_r2i1p1_path_list,parallel=True)
CCiA_HadGEM2_CC_historical_psl_r2i1p1_ds = CCiA_HadGEM2_CC_historical_psl_r2i1p1_ds.rename({'psl':'psl_r2i1p1'})

In [87]:
CCiA_HadGEM2_CC_historical_psl_r3i1p1_ds = xr.open_mfdataset(CCiA_HadGEM2_CC_historical_psl_r3i1p1_path_list,parallel=True)
CCiA_HadGEM2_CC_historical_psl_r3i1p1_ds = CCiA_HadGEM2_CC_historical_psl_r3i1p1_ds.rename({'psl':'psl_r3i1p1'})

In [121]:
CCiA_HadGEM2_CC_historical_psl_ds =  xr.merge([CCiA_HadGEM2_CC_historical_psl_r1i1p1_ds,CCiA_HadGEM2_CC_historical_psl_r2i1p1_ds,CCiA_HadGEM2_CC_historical_psl_r3i1p1_ds])

### make ensemble axis and calculate e-mean

In [123]:
CCiA_HadGEM2_CC_historical_psl_ds

<xarray.Dataset>
Dimensions:     (bnds: 2, lat: 145, lon: 192, time: 553)
Coordinates:
  * time        (time) object 1959-12-16 00:00:00 ... 2005-12-16 00:00:00
  * lat         (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon         (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds   (time, bnds) object dask.array<chunksize=(300, 2), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(300, 145, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(300, 192, 2), meta=np.ndarray>
    psl_r1i1p1  (time, lat, lon) float32 dask.array<chunksize=(300, 145, 192), meta=np.ndarray>
    psl_r2i1p1  (time, lat, lon) float32 dask.array<chunksize=(300, 145, 192), meta=np.ndarray>
    psl_r3i1p1  (time, lat, lon) float32 dask.array<chunksize=(300, 145, 192), meta=np.ndarray>

In [137]:
CCiA_HadGEM2_CC_historical_psl_da = xr.concat([CCiA_HadGEM2_CC_historical_psl_ds.psl_r1i1p1, CCiA_HadGEM2_CC_historical_psl_ds.psl_r2i1p1, CCiA_HadGEM2_CC_historical_psl_ds.psl_r3i1p1], "ensemble")
CCiA_HadGEM2_CC_historical_psl_da = CCiA_HadGEM2_CC_historical_psl_da.rename('psl')

In [141]:
CCiA_HadGEM2_CC_historical_psl = CCiA_HadGEM2_CC_historical_psl_da.mean(dim='ensemble')
CCiA_HadGEM2_CC_historical_psl

<xarray.DataArray 'psl' (time: 553, lat: 145, lon: 192)>
dask.array<mean_agg-aggregate, shape=(553, 145, 192), dtype=float32, chunksize=(300, 145, 192), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1959-12-16 00:00:00 ... 2005-12-16 00:00:00
  * lat      (lat) float64 -90.0 -88.75 -87.5 -86.25 ... 86.25 87.5 88.75 90.0
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1

In [34]:
CCiA_HadGEM2_CC_historical_psl_catalog.search(date_range='198412-200511').unique('path')['path']['values']

['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/historical/mon/atmos/Amon/r1i1p1/v20110927/psl/psl_Amon_HadGEM2-CC_historical_r1i1p1_198412-200511.nc']

In [24]:
num_historical_date_range = CCiA_HadGEM2_CC_historical_catalog.unique('date_range')['date_range']['count']
historical_date_range_list = CCiA_HadGEM2_CC_historical_catalog.unique('date_range')['date_range']['values']

In [57]:
CCiA_HadGEM2_CC_rcp85_catalog.unique('path')

{'path': {'count': 26,
  'values': ['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_203012-205511.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_205512-208011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_208012-209912.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_210001-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/

> do we have ensembles? how many? how many versions? how many date ranges for each experiment?

In [38]:
CCiA_HadGEM2_CC_rcp85_catalog.unique('ensemble')

{'ensemble': {'count': 3, 'values': ['r1i1p1', 'r2i1p1', 'r3i1p1']}}

In [39]:
CCiA_HadGEM2_CC_historical_catalog.unique('ensemble')

{'ensemble': {'count': 3, 'values': ['r1i1p1', 'r2i1p1', 'r3i1p1']}}

In [40]:
CCiA_HadGEM2_CC_rcp85_catalog.unique('version')

{'version': {'count': 3, 'values': ['v20111215', 'v20120105', 'v20120531']}}

In [41]:
CCiA_HadGEM2_CC_historical_catalog.unique('version')

{'version': {'count': 3, 'values': ['v20110927', 'v20111129', 'v20111208']}}

In [42]:
CCiA_HadGEM2_CC_historical_catalog.unique('date_range')

{'date_range': {'count': 7,
  'values': ['185912-188411',
   '188412-190911',
   '190912-193411',
   '193412-195911',
   '195912-198411',
   '198412-200511',
   '198412-200512']}}

In [43]:
CCiA_HadGEM2_CC_rcp85_catalog.unique('date_range')

{'date_range': {'count': 5,
  'values': ['200512-203011',
   '203012-205511',
   '205512-208011',
   '208012-209912',
   '210001-210012']}}

#### we need to build 6 datasets and ensemble average into one historical and one rcp85 dataset

In [53]:
CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog = NCI_catalog.esgf.cmip5_all.search\
(institute="MOHC",variable=['psl','pr'],time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',ensemble='r1i1p1')
CCiA_HadGEM2_CC_rcp85_r2i1p1_catalog = NCI_catalog.esgf.cmip5_all.search\
(institute="MOHC",variable=['psl','pr'],time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',ensemble='r2i1p1')
CCiA_HadGEM2_CC_rcp85_r3i1p1_catalog = NCI_catalog.esgf.cmip5_all.search\
(institute="MOHC",variable=['psl','pr'],time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',ensemble='r3i1p1')

In [47]:
CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('version')['version']['count']

1

In [48]:
CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('path')

{'path': {'count': 10,
  'values': ['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_203012-205511.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_205512-208011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_208012-209912.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/pr/pr_Amon_HadGEM2-CC_rcp85_r1i1p1_210001-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/

In [50]:
CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('date_range')['date_range']['count']

5

In [51]:
CCiA_HadGEM2_CC_rcp85_r1i1p1_ds = xr.open_mfdataset(CCiA_HadGEM2_CC_rcp85_r1i1p1_catalog.unique('path')['path']['values'])

In [11]:
CCiA_ACCESS1_0 = NCI_catalog.esgf.cmip5_all.search(institute="CSIRO-BOM",variable='psl',time_frequency='mon',experiment='rcp85',model='ACCESS1.0',version='v20120727')

In [12]:
NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)').unique('path')

{'path': {'count': 5,
  'values': ['/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r1i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r1i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r2i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r2i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20140129/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20140310/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc']}}

In [13]:
CCiA_CESM1_CAM5_r1i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r1i1p1',version='v20130313')

In [14]:
CCiA_CESM1_CAM5_r2i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r2i1p1',version='v20130313')

In [17]:
CCiA_CESM1_CAM5_r3i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r3i1p1',version='v20140310')

In [ ]:
CCiA_CESM1_CAM5_r3i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r3i1p1',version='v20140310')

In [21]:
NCI_catalog.esgf.cmip5_all.search(institute="CNRM-CERFACS",variable='psl',time_frequency='mon',experiment='rcp85',model='CNRM-CM5',ensemble='r10i1p1').unique('path')

{'path': {'count': 2,
  'values': ['/g/data/al33/replicas/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/mon/atmos/Amon/r10i1p1/v20110915/psl/psl_Amon_CNRM-CM5_rcp85_r10i1p1_200601-205512.nc',
   '/g/data/al33/replicas/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/mon/atmos/Amon/r10i1p1/v20110915/psl/psl_Amon_CNRM-CM5_rcp85_r10i1p1_205601-210012.nc']}}

In [63]:
NCI_catalog.esgf.cmip5_all.search(variable='psl',time_frequency='mon',experiment='rcp85').unique('institute')

{'institute': {'count': 23,
  'values': ['BCC',
   'BNU',
   'CCCMA',
   'CMCC',
   'CNRM-CERFACS',
   'CSIRO-BOM',
   'CSIRO-QCCCE',
   'FIO',
   'ICHEC',
   'INM',
   'IPSL',
   'LASG-CESS',
   'LASG-IAP',
   'MIROC',
   'MOHC',
   'MPI-M',
   'MRI',
   'NASA-GISS',
   'NCAR',
   'NCC',
   'NIMR-KMA',
   'NOAA-GFDL',
   'NSF-DOE-NCAR']}}

In [64]:
NCI_catalog.esgf.cmip5_all.search(variable='psl',time_frequency='mon',experiment='rcp85').unique('model')

{'model': {'count': 42,
  'values': ['ACCESS1.0',
   'ACCESS1.3',
   'BCC-CSM1.1',
   'BCC-CSM1.1(m)',
   'BNU-ESM',
   'CCSM4',
   'CESM1(BGC)',
   'CESM1(CAM5)',
   'CESM1(WACCM)',
   'CMCC-CESM',
   'CMCC-CM',
   'CMCC-CMS',
   'CNRM-CM5',
   'CSIRO-Mk3L-1-2',
   'CanESM2',
   'EC-EARTH',
   'FGOALS-g2',
   'FGOALS-s2',
   'FIO-ESM',
   'GFDL-CM3',
   'GFDL-ESM2G',
   'GFDL-ESM2M',
   'GISS-E2-H',
   'GISS-E2-H-CC',
   'GISS-E2-R',
   'GISS-E2-R-CC',
   'HadGEM2-AO',
   'HadGEM2-CC',
   'HadGEM2-ES',
   'INM-CM4',
   'IPSL-CM5A-LR',
   'IPSL-CM5A-MR',
   'IPSL-CM5B-LR',
   'MIROC-ESM',
   'MIROC-ESM-CHEM',
   'MIROC5',
   'MPI-ESM-LR',
   'MPI-ESM-MR',
   'MRI-CGCM3',
   'MRI-ESM1',
   'NorESM1-M',
   'NorESM1-ME']}}

In [9]:
NCI_catalog.esgf.search(variable='psl').unique('path')

TypeError: search() got an unexpected keyword argument 'variable'

In [30]:
result = cordex.search(variable='psl')
result

,unique
project,1
product,1
domain,2
institute,3
driving_model,12
experiment,4
ensemble,1
rcm_name,7
rcm_version,2
time_frequency,5


In [39]:
cordex.search(institute='UNSW', time_frequency='day', variable='psl', domain='AUS-44i', require_all_on=['rcm_name']).unique('path')

{'path': {'count': 66,
  'values': ['/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19510101-19551231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19560101-19601231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19610101-19651231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19660101-19701231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/histori

# $The\ End$

In [16]:
cluster.close()
client.close()